In [1]:
!pip install -q ultralytics roboflow==1.* opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 109.9 MB/s eta 0:00:00


In [2]:
import torch
print("GPU disponible:", torch.cuda.is_available())
!nvidia-smi

GPU disponible: True
Wed Aug 27 12:28:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="8J9CgvKgWtTVxnEYRIuI")
project = rf.workspace("computervisionfactoria").project("factoria-f5-citpf")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Factoria-F5-3 in yolov8:: 100%|██████████| 4448/4448 [00:00<00:00, 5192.00it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
from ultralytics import YOLO
import os

data_yaml = os.path.join(dataset.location, "data.yaml")
model = YOLO("yolov8n.pt")  # empieza con n (nano); luego puedes probar s o m

results = model.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    device=0,          # <--- 0 = usa la primera GPU
    name="factoria_f5_gpu"
)

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Factoria-F5-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=factoria_f5_gpu, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

In [6]:
metrics = model.val(data=data_yaml, split="test", imgsz=640)
print(metrics)

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1217.4±591.4 MB/s, size: 73.7 KB)
val: Scanning /content/Factoria-F5-3/test/labels... 145 images, 21 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 145/145 2581.6it/s 0.1s
val: New cache created: /content/Factoria-F5-3/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 3, len(boxes) = 199. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 3.7it/s 2.7s
                   all        145        199       0.85       0.73      0.779      0.446
                    F5          5          7      0.598      0.286   

In [7]:
pred = model.predict(
    source=os.path.join(dataset.location, "test", "images"),
    conf=0.25,
    imgsz=640,
    save=True,
    project="runs/predict",
    name="factoria_f5_demo"
)
print("Resultados guardados en:", pred[0].save_dir)


image 1/145 /content/Factoria-F5-3/test/images/1656606217870_jpg.rf.40a9549e59c1e7130025da33ade1564d.jpg: 640x640 (no detections), 7.3ms
image 2/145 /content/Factoria-F5-3/test/images/1665735788978_jpg.rf.eb4ec9636103963702fcc9b6e669893e.jpg: 640x640 1 Factoria, 7.3ms
image 3/145 /content/Factoria-F5-3/test/images/1665735790387_jpg.rf.3bac271f62285bca1c9f7d4fac764ee1.jpg: 640x640 1 Factoria, 1 FemCoders, 7.3ms
image 4/145 /content/Factoria-F5-3/test/images/1667459436404_jpg.rf.cd3276ef287a94fe72f3feff7b13bac9.jpg: 640x640 1 Factoria, 7.3ms
image 5/145 /content/Factoria-F5-3/test/images/1737560543054_jpg.rf.6177b185de8a332d0380a55a3cf13ed3.jpg: 640x640 1 Factoria, 1 Fundacion Orange, 1 SomosF5, 7.3ms
image 6/145 /content/Factoria-F5-3/test/images/1742225856914_jpg.rf.1de064c1ad7f9d76d7197f85691debd0.jpg: 640x640 1 Factoria, 1 SomosF5, 7.3ms
image 7/145 /content/Factoria-F5-3/test/images/225259028_579436796797107_1163772387798740733_n_jpg.rf.1182a103377a2683400ce4b731969f60.jpg: 640x640